In [ ]:
#The user need to copy this script to corresponding folder and then run the corresponding code cell

In [ ]:
from Bio import PDB
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

def get_energy(file_path, ab_start, ab_end, ag_start, ag_end):
    """
    Calculate the energy contributions of residues within specified ranges.

    Parameters:
        file_path (str): Path to the file containing energy contributions.
        ab_start (int): Antibody start residue ID.
        ab_end (int): Antibody end residue ID.
        ag_start (int): Antigen start residue ID.
        ag_end (int): Antigen end residue ID.

    Returns:
        pandas.DataFrame: DataFrame containing residue IDs and their energy contributions.
    """
    
    # Read the data file
    res_ctr_df = pd.read_csv(file_path, delim_whitespace=True)
    columns = list(res_ctr_df.columns)
    new_columns = columns[1:] + [None]
    res_ctr_df.columns = new_columns
    res_ctr_df = res_ctr_df.drop([None], axis=1)

    # Filter data based on specified ranges
    df = res_ctr_df
    df = df[(df['ID1'] >= ab_start) & (df['ID1'] <= ab_end)]
    df = df[(df['ID2'] >= ag_start) & (df['ID2'] <= ag_end)]

    # Sum up data for each residue
    df = df.drop(['ID2'], axis=1)
    df = df.groupby('ID1').sum().reset_index()
    score_df = df
    return score_df


def get_seq(pdb_file, chain_ID):
    """
    Extract amino acid sequence from a PDB file for a given chain.

    Parameters:
        pdb_file (str): Path to the PDB file.
        chain_ID (str): Chain ID.

    Returns:
        list: List of tuples containing amino acid names and residue numbers.
    """
    parser = PDB.PDBParser(QUIET=True)
    structure = parser.get_structure('pdb_structure', pdb_file)

    amino_acids = []
    for model in structure:
        for chain in model:
            if chain.id == chain_ID:
                for residue in chain:
                    # Include information for all residues, including standard and non-standard amino acids.
                    amino_acids.append((residue.get_resname(), residue.id[1]))
    return amino_acids


def parse_pdb(chain_ID):
    """
    Parse PDB files in the current directory to obtain amino acid sequences.

    Parameters:
        chain_ID (str): Chain ID.

    Returns:
        pandas.DataFrame: DataFrame containing residue names and numbers.
    """
    # Load each pdb file
    files = sorted([f for f in os.listdir('.') if f.endswith('.pdb')])
    for f in files:
        amino_acids = get_seq(f, chain_ID)
        # Create a DataFrame to store the amino acids
        seq_df = pd.DataFrame(amino_acids, columns=['Residue', 'Number'])
    return seq_df


def dr_sv_barplot_Nb60(seq_df, score_df, show_top_res_number):
    """
    Draw and save a bar plot showing the energy contributions of residues.

    Parameters:
        seq_df (pandas.DataFrame): DataFrame containing residue names and numbers.
        score_df (pandas.DataFrame): DataFrame containing residue IDs and energy contributions.
        show_top_res_number (int): Number of top residues to show in the plot.
    """
    if len(seq_df) == len(score_df):
        print("Two DataFrames have consistent rows, continue")
        # Only for this Nb60 MD result!!!
        score_df['Res_ID_renew'] = score_df['ID1'].sub(237)
        merged_df = pd.concat([score_df, seq_df['Residue']], axis=1)
        df = merged_df
        df = df[df['MMGBSA'] <= 0]
        df.loc[:, "Res_ID_renew"] = df["Res_ID_renew"].astype(int)
        df = df.reset_index(drop=True)
        df.index = df.index + 1
        df['ResName'] = df.apply(lambda x: str(x['Residue']) + str(x['Res_ID_renew']), axis=1)

        # Next, draw the plot
        df_asc = df.sort_values(by=['MMGBSA'], ascending=True)
        df_asc = df_asc.head(show_top_res_number)
        df_asc['Res_ID_renew'] = df_asc['Res_ID_renew'].astype(str)

        output_df = df_asc[["ResName", "MMGBSA"]]
        output_df.loc[:, "MMGBSA"] = round(output_df["MMGBSA"], 4)
        output_df.to_csv("Residue_contribution_rank.csv", index=False)

        palette = sns.cubehelix_palette(show_top_res_number + 5, as_cmap=False, gamma=.5, reverse=True, dark=.15, light=.75)
        fig, ax = plt.subplots(figsize=(10, 4))
        sns.barplot(x='ResName', y='MMGBSA', data=df_asc, saturation=1, width=0.7, edgecolor='0', linewidth=0.5, palette=palette)
        ax.set_xlabel('')
        ax.set_ylabel('MMGBSA (kcal/mol)')
        ax.set_title('')
        ax.bar_label(ax.containers[0], rotation=90, fmt='%.2f')

        ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

        plt.ylim(-28, 0)
        sns.despine(top=False, right=True, bottom=True)
        files = sorted([f for f in os.listdir('.') if f.endswith('.pdb')])
        plt.savefig(files[0] + "_.png", dpi=300, bbox_inches="tight")
        plt.show()
    else:
        raise ValueError("The number of rows in the two DataFrames is inconsistent, and the program terminates!")


def dr_sv_barplot(seq_df, score_df, show_top_res_number):
    """
    Draw and save a bar plot showing the energy contributions of residues.

    Parameters:
        seq_df (pandas.DataFrame): DataFrame containing residue names and numbers.
        score_df (pandas.DataFrame): DataFrame containing residue IDs and energy contributions.
        show_top_res_number (int): Number of top residues to show in the plot.
    """
    if len(seq_df) == len(score_df):
        print("Two DataFrames have consistent rows, continue")

        merged_df = pd.concat([score_df, seq_df['Residue']], axis=1)

        df = merged_df
        df = df[df['MMGBSA'] <= 0]
        df.loc[:, "ID1"] = df["ID1"].astype(int)
        df = df.reset_index(drop=True)
        df.index = df.index + 1
        df['ResName'] = df.apply(lambda x: str(x['Residue']) + str(x['ID1']), axis=1)

        df_asc = df.sort_values(by=['MMGBSA'], ascending=True)
        df_asc = df_asc.head(show_top_res_number)
        df_asc['ID1'] = df_asc['ID1'].astype(str)

        output_df = df_asc[["ResName","MMGBSA"]]
        output_df.loc[:, "MMGBSA"] = round(output_df["MMGBSA"], 4)
        output_df.to_csv("Residue_contribution_rank.csv", index=False)

        palette = sns.cubehelix_palette(show_top_res_number + 5, as_cmap=False, gamma=.5, reverse=True, dark=.15, light=.75)
        fig, ax = plt.subplots(figsize=(10, 4))
        sns.barplot(x='ResName', y='MMGBSA', data=df_asc, saturation=1, width=0.7, edgecolor='0', linewidth=0.5, palette=palette)
        ax.set_xlabel('')
        ax.set_ylabel('MMGBSA (kcal/mol)')
        ax.set_title('')
        ax.bar_label(ax.containers[0], rotation=90, fmt='%.2f')

        ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

        plt.ylim(-28, 0)
        sns.despine(top=False, right=True, bottom=True)
        files = sorted([f for f in os.listdir('.') if f.endswith('.pdb')])
        plt.savefig(files[0] + "_.png", dpi=300, bbox_inches="tight")
        plt.show()
    else:
        raise ValueError("The number of rows in the two DataFrames is inconsistent, and the program terminates!")


In [ ]:
#Run code cell for Nb60 binding pose
chain_ID = "B"
ab_start = 238
ab_end = 370
ag_start = 1
ag_end = 237
show_top_res_number = 20
file_path = "ie_ave.dat"
# Calculate energy contributions and plot the bar plot and summarize total MMGBSA
score_df = get_energy(file_path,ab_start,ab_end,ag_start,ag_end)
seq_df = parse_pdb(chain_ID)
dr_sv_barplot_Nb60(seq_df,score_df,show_top_res_number)
sum_MMGBSA = score_df['MMGBSA'].sum()
print("The total MMGBSA = " + str(sum_MMGBSA))

In [ ]:
#Run code cell for Nb108 binding pose
chain_ID = "H"
ab_start = 1
ab_end = 122
ag_start = 123
ag_end = 359
show_top_res_number = 20
file_path = "ie_ave.dat"
# Calculate energy contributions and plot the bar plot and summarize total MMGBSA
score_df = get_energy(file_path,ab_start,ab_end,ag_start,ag_end)
seq_df = parse_pdb(chain_ID)
dr_sv_barplot(seq_df,score_df,show_top_res_number)
sum_MMGBSA = score_df['MMGBSA'].sum()
print("The total MMGBSA = " + str(sum_MMGBSA))

In [ ]:
#Run code cell for Nb65 binding pose
chain_ID = "H"
ab_start = 1
ab_end = 126
ag_start = 127
ag_end = 362
show_top_res_number = 20
file_path = "ie_ave.dat"
# Calculate energy contributions and plot the bar plot and summarize total MMGBSA
score_df = get_energy(file_path,ab_start,ab_end,ag_start,ag_end)
seq_df = parse_pdb(chain_ID)
dr_sv_barplot(seq_df,score_df,show_top_res_number)
sum_MMGBSA = score_df['MMGBSA'].sum()
print("The total MMGBSA = " + str(sum_MMGBSA))

In [ ]:
#The following code is used to generate RMSD line plot 

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def draw_line_plot(file_path):
    """
    Draw a line plot for data in the given file and save it.

    Parameters:
    - file_path (str): Path to the input data file.

    Returns:
    - None
    """
    # Extract abbreviation from file path
    abbreviation = os.path.basename(file_path)[:10]

    # Prepare save name for the plot
    save_name = file_path + '_lineplot.png'

    # Read data from file
    try:
        df = pd.read_csv(file_path, delim_whitespace=True, header=0)
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return

    # Convert frame to nanoseconds
    df['#Frame'] = df['#Frame'] / 10

    # Create the plot
    fig, ax = plt.subplots(figsize=(10, 4), subplot_kw={'box_aspect': 1.8 / 5})
    lw_value = 0.7
    sns.lineplot(data=df, x='#Frame', y=df.columns[2], label=df.columns[2], linewidth=lw_value, color='#f18783')
    sns.lineplot(data=df, x='#Frame', y=df.columns[1], label=df.columns[1], linewidth=lw_value, color='#f7b361')

    # Set plot limits and ticks
    x_max = 500
    y_max = 6.3
    ax.set_xlim(0, x_max)
    ax.set_ylim(0, y_max)
    ax.xaxis.tick_bottom()
    ax.yaxis.tick_left()
    ax.set_xticks(np.arange(0, x_max + 1, 20))
    ax.set_yticks(np.arange(0, y_max + 1, 1))
    ax.spines[['right', 'top']].set_visible(False)

    # Set axis labels
    plt.xlabel('Simulation Time (ns)')
    plt.ylabel('RMSD (Å)')

    # Show legend
    plt.legend(loc='upper left', title_fontsize='medium', shadow=True, edgecolor='black', framealpha=1.0, ncol=1, mode=None)
    plt.title("")

    # Save and show plot
    plt.savefig(save_name, dpi=300, bbox_inches='tight')
    plt.show()

def draw_line_plots_in_directory(directory="."):
    """
    Draw line plots for all files ending with "rmsd.dat" in the given directory.

    Parameters:
    - directory (str): Path to the directory containing data files.

    Returns:
    - None
    """
    for filename in os.listdir(directory):
        if filename.endswith("rmsd.dat"):
            file_path = os.path.join(directory, filename)
            draw_line_plot(file_path)

# Usage
draw_line_plots_in_directory()
